In [20]:
import sqlite3

In [21]:
# connect to the database 
con = sqlite3.connect('./test.db')
# create a cursor object
cur = con.cursor()

In [22]:
# show content of database
cur.execute("SELECT name FROM sqlite_master")
print(cur.fetchall())

[]


In [23]:
# to link tables
cur.execute("PRAGMA foreign_keys = ON;")
con.commit()
cur.execute("PRAGMA foreign_keys;")
print(cur.fetchall())

[(1,)]


In [24]:
# create table that does not allow duplicates
#cur.execute("DROP TABLE IF EXISTS simulations")
cur.execute("""
CREATE TABLE IF NOT EXISTS simulations (
            simulation_id integer PRIMARY KEY,
            N int NOT NULL,
            K int NOT NULL,
            lambda float NOT NULL,
            mu float NOT NULL,
            h float NOT NULL,
            seed int NOT NULL,
            raw_file TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX parameters ON simulations (N,K,lambda,mu,h,seed);")
# can be dropped with cur.execute("DROP INDEX parameters;") if needs to be redesigned
con.commit()

In [25]:
cur.execute("INSERT INTO simulations (N, K, lambda, mu, h, seed, raw_file) VALUES (?, ?, ?, ?, ?, ?, ?)", (10000,10,1-1e-8,0.5,0.5,1,'./test.csv'))
con.commit()

In [26]:
cur.execute("SELECT * FROM simulations")
print(cur.fetchall())

[(1, 10000, 10, 0.99999999, 0.5, 0.5, 1, './test.csv')]


In [27]:
def insert_simulation(params):
    # the sql command does not specify the values but only placeholders with ?
    sql=f"INSERT INTO simulations ({','.join(params.keys())}) VALUES ({','.join(['?']*len(params.keys()))})"
    # values are passed when executing the command
    cur.execute(sql, list(params.values()))
    con.commit()


# add an example simulation to the table 
params={'N':1000, 'K':100, 'lambda':0.9, 'mu':0.2, 'h':0.01, 'seed':1000, 'raw_file':'path'}
insert_simulation(params)
# add another example simulation to the table
params['lambda']=0.99
insert_simulation(params)

In [28]:
cur.execute("SELECT * FROM simulations")
cur.fetchall()

[(1, 10000, 10, 0.99999999, 0.5, 0.5, 1, './test.csv'),
 (2, 1000, 100, 0.9, 0.2, 0.01, 1000, 'path'),
 (3, 1000, 100, 0.99, 0.2, 0.01, 1000, 'path')]

In [29]:
try:
    # this cannot be added because it is already in the database
    cur.execute("INSERT INTO simulations (N,K,lambda,mu,h,seed,raw_file) VALUES (1000, 100, 0.99, 0.2, 0.01, 1000, 'path')")
except sqlite3.IntegrityError:
    print("This simulation is already in the database")
con.commit()
cur.execute("SELECT * FROM simulations")
print(cur.fetchall())

This simulation is already in the database
[(1, 10000, 10, 0.99999999, 0.5, 0.5, 1, './test.csv'), (2, 1000, 100, 0.9, 0.2, 0.01, 1000, 'path'), (3, 1000, 100, 0.99, 0.2, 0.01, 1000, 'path')]


In [30]:
try:
    cur.execute("INSERT INTO simulations (N) VALUES (1000)")
except sqlite3.IntegrityError:
    print("IntegrityError because not all parameters are given")


IntegrityError because not all parameters are given


In [31]:
cur.execute("DROP TABLE IF EXISTS beta_approximations")
# add another table with beta approximation
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_approximations (
            simulation_id INTEGER NOT NULL UNIQUE,
            a float NOT NULL,
            b float NOT NULL,
            loc float NOT NULL,
            scale float NOT NULL,
            FOREIGN KEY (simulation_id) REFERENCES simulations (simulation_id)
            )
""")
con.commit()
cur.execute("SELECT name FROM sqlite_master")
print(cur.fetchall())


[('simulations',), ('parameters',), ('beta_approximations',), ('sqlite_autoindex_beta_approximations_1',)]


In [32]:
# get simulation ID from simulations table
params = {'N':1000, 'K':100, 'lambda':0.9, 'mu':0.2, 'h':0.01, 'seed':1000, 'raw_file':'path'}
cur.execute("SELECT simulation_id FROM simulations WHERE N=:N AND K=:K AND lambda=:lambda AND mu=:mu AND h=:h AND seed=:seed AND raw_file=:raw_file", params)
simulation_id=cur.fetchone()[0]
beta_approximation={'a':0.9, 'b':0.2, 'loc':0.01, 'scale':1000}
cur.execute(f"INSERT INTO beta_approximations (a,b,loc,scale, simulation_id) VALUES (0.9, 0.2, 0.01, 1000, {simulation_id})")
con.commit()
cur.execute("SELECT * FROM beta_approximations")
print(cur.fetchall())

[(2, 0.9, 0.2, 0.01, 1000.0)]


In [33]:
# update value in approximation
cur.execute("UPDATE beta_approximations SET a=0.99 WHERE simulation_id=1")
con.commit()
cur.execute("SELECT * FROM beta_approximations")
print(cur.fetchall())

[(2, 0.9, 0.2, 0.01, 1000.0)]


In [34]:
# add a new simulation
simulation={'N':1000, 'K':100, 'lambda':0.0, 'mu':0.2, 'h':0.01, 'seed':1000, 'raw_file':'path'}
insert_simulation(simulation)
cur.execute("SELECT * FROM simulations")
cur.fetchall()

[(1, 10000, 10, 0.99999999, 0.5, 0.5, 1, './test.csv'),
 (2, 1000, 100, 0.9, 0.2, 0.01, 1000, 'path'),
 (3, 1000, 100, 0.99, 0.2, 0.01, 1000, 'path'),
 (4, 1000, 100, 0.0, 0.2, 0.01, 1000, 'path')]

In [35]:
# close database
con.close()
# delete file
import os
os.remove('./test.db')